In [179]:
from pydantic import BaseModel
from transformers import AutoTokenizer
import requests, os, datetime, json


def structuredOutput(extraction_target: str, extraction_schema: BaseModel, reasoning: bool = True):
    url = "https://api.friendli.ai/serverless/v1/completions"
    token = os.getenv("FRIENDLI_TOKEN") or "<YOUR_FRIENDLI_TOKEN>"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    extraction_system_prompt = f"""Extract the information.
    follow the schema: {extraction_schema.model_json_schema()}
    """

    messages = [
        {
            "content": extraction_system_prompt,
            "role": "system"
        },
        {
            "content": extraction_target,
            "role": "user"
        },
    ]

    if reasoning:
        reasoning_messages = [
            *messages,
            {
                "role": "assistant",
                "content": "<think>\n",
            },
        ]

        prompt = AutoTokenizer.from_pretrained(
            "deepseek-ai/deepseek-v3",
            token=os.environ.get("HF_TOKEN"),
            legacy=False,
        ).apply_chat_template(reasoning_messages, tokenize=False, continue_final_message=True)
        
        reasoning = requests.request("POST", url, headers=headers,
            json={
                "model": "deepseek-r1",
                "prompt": prompt,
                "stop": ["</think>"],
            }).json()['choices'][0]['text']
    else:
        reasoning = ""

    final_messages = [
        *messages,
        {
            "role": "assistant",
            "content": "<think>\n" + reasoning + "\n</think>\n"
        },
    ]

    prompt = AutoTokenizer.from_pretrained(
        "deepseek-ai/deepseek-v3",
        token=os.environ.get("HF_TOKEN"),
        legacy=False,
    ).apply_chat_template(final_messages, tokenize=False, continue_final_message=True)

    response = requests.request("POST", url, headers=headers,
        json={
            "model": "deepseek-r1",
            "prompt": prompt,
            "response_format": {
                "type": "json_schema",
                "json_schema": {
                    "schema": extraction_schema.model_json_schema(),
                }
            }
        })
    raw_json = response.json()['choices'][0]['text']
    parsed = json.loads(raw_json) if isinstance(raw_json, str) else raw_json

    return parsed, reasoning

In [183]:
class extraction_schema(BaseModel):
    name: str
    date: datetime.date
    participants: list[str]

extraction_target = f"""Alice and Bob are going to a science fair on Friday.
today is {datetime.date.today()}
"""

parsed, reasoning = structuredOutput(
    extraction_target,
    extraction_schema,
    reasoning=True
)

# print(reasoning)
print(json.dumps(parsed, indent=2))

{
  "name": "science fair",
  "date": "2025-05-09",
  "participants": [
    "Alice",
    "Bob"
  ]
}


In [185]:
from typing import Annotated, Optional
import json
from function_schema import get_function_schema

def get_weather(
    city: Annotated[str, "The city to get the weather for"],
    unit: Annotated[Optional[str], "The unit to return the temperature in"] = "celcius",
) -> str:
    """Returns the weather for the given city."""
    return f"Weather for {city} is 20°C"

def get_news(
    topic: Annotated[str, "The topic to get news for"],
    source: Annotated[Optional[str], "The source to get news from"] = None,
) -> str:
    """Returns the news for the given topic."""
    return f"News for {topic} from {source if source else 'all sources'}"

tools = [
    get_weather,
    get_news,
]

tool_schemas = [get_function_schema(tool) for tool in tools]
print(json.dumps(tool_schemas, indent=2))

[
  {
    "name": "get_weather",
    "description": "Returns the weather for the given city.",
    "parameters": {
      "type": "object",
      "properties": {
        "city": {
          "type": "string",
          "description": "The city to get the weather for"
        },
        "unit": {
          "type": "string",
          "description": "The unit to return the temperature in",
          "default": "celcius"
        }
      },
      "required": [
        "city"
      ]
    }
  },
  {
    "name": "get_news",
    "description": "Returns the news for the given topic.",
    "parameters": {
      "type": "object",
      "properties": {
        "topic": {
          "type": "string",
          "description": "The topic to get news for"
        },
        "source": {
          "type": "string",
          "description": "The source to get news from",
          "default": null
        }
      },
      "required": [
        "topic"
      ]
    }
  }
]


In [187]:
from pydantic import BaseModel, Field, ValidationError
from typing import List, Dict, Any, Optional, Union, Type, Tuple

class ToolCallInternal(BaseModel):
    tool_name: str = Field(..., description="호출할 도구의 이름")
    arguments: Dict[str, Any] = Field(..., description="도구 호출에 필요한 인자들 (딕셔너리 형태)")

class Blueprint(BaseModel):
    q: str = Field(..., description="사용자의 초기 질문 또는 요청")
    a_gt: List[ToolCallInternal] = Field(..., description="Ground Truth: 에이전트가 수행해야 할 도구 호출 순서")
    o_gt: str = Field(..., description="예상되는 최종 결과에 대한 설명 (문자열)")

system_prompt_gen = f"""당신은 APIGen-MT 파이프라인의 1단계를 위한 '작업 청사진 생성자'입니다. 당신의 목표는 사용자와 AI 에이전트 간의 **현실적이고 검증 가능한 다중 턴 상호작용 시나리오**를 나타내는 상세한 청사진을 만드는 것입니다. 주어진 도구 설명을 바탕으로 다음 구성요소를 포함하는 **JSON 객체**를 생성해야 합니다.

1.  `q` (string): 사용자의 초기 질문/요청입니다. **구체적이고 자연스러워야 하며**, 에이전트가 정보를 조회하고, 상태를 변경하고, 사용자에게 다시 확인하는 등 **여러 단계의 상호작용이 필요할 수 있는** 시나리오를 나타내는 것이 좋습니다.
2.  `a_gt` (list): 사용자의 요청 `q`를 **완전히, 그리고 정확한 순서대로** 해결하기 위해 에이전트가 호출해야 하는 **Ground Truth 도구 호출 리스트**입니다. 각 요소는 `{{"tool_name": "도구명", "arguments": {{"인자명": "값", ...}}}}` 형식이어야 합니다. 사용 가능한 도구 설명과 반드시 일치해야 합니다. **최소 1개 이상의 도구 호출**을 포함해야 하며, 가능하면 **2개 이상의 순차적 도구 호출**이 필요한 시나리오를 만드세요.
3.  `o_gt` (string): 모든 `a_gt`가 성공적으로 실행되었다고 가정했을 때, 에이전트가 사용자에게 최종적으로 제공해야 할 **결과 요약 또는 응답 메시지에 대한 자연스러운 설명**입니다. `a_gt`의 모든 결과를 반영해야 합니다.

**사고 과정 (Optional but Recommended):**
- 최종 JSON을 생성하기 전에, 당신의 **사고 과정 (어떤 시나리오를 구상했고, 왜 특정 도구와 순서를 선택했는지 등)을 `<think>...</think>` 태그 안에 작성**할 수 있습니다.

**응답 규칙:**
- 최종 응답에는 반드시 위에 설명된 구조를 가진 **유효한 JSON 객체**가 ```json ... ``` 코드 블록 안에 포함되어야 합니다.
- 코드 블록 앞이나 뒤에 **JSON 객체 외의 다른 텍스트는 포함하지 마십시오** (단, 먼저 나오는 `<think>` 블록은 허용됩니다)."""
user_prompt_gen = f"사용 가능한 도구:\n{tool_schemas}\n\n위 도구들을 사용하는 시나리오를 위한 JSON 객체를 위의 요구사항과 규칙에 맞게 생성해주세요."

parsed, reasoning = structuredOutput(
    extraction_target=f"""{system_prompt_gen}
    {user_prompt_gen}""",
    extraction_schema=Blueprint,
    reasoning=True
)

# print(reasoning)
print(json.dumps(parsed, indent=2, ensure_ascii=False))


{
  "q": "서울의 현재 날씨와 최근 주요 뉴스를 알려주세요.",
  "a_gt": [
    {
      "tool_name": "get_weather",
      "arguments": {
        "city": "Seoul"
      }
    },
    {
      "tool_name": "get_news",
      "arguments": {
        "topic": "Seoul"
      }
    }
  ],
  "o_gt": "서울의 현재 날씨는 22°C로 맑음 상태입니다. 최신 뉴스로는 1) 서울시 도시 재생 프로젝트 확대 2) 관광객 증가에 따른 인프라 정비 계획 발표 등이 있습니다."
}
